In [2]:
with open('input.txt', 'r') as file:
    text = file.read()

print("length of dataset in charcters: ", len(text))


length of dataset in charcters:  1115394


In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print('vocab size: ', vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size:  65


In [4]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[ch] for ch in s] # encode: takes a string, outputs a list of integrs
decode = lambda l: ''.join([itos[i] for i in l]) # decode: takes a list of integers, outputs a string   

print(encode('hii there'))
print(decode(encode('hii there')))


[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [1]:
!pip install torch

zsh:1: /opt/homebrew/bin/pip: bad interpreter: /opt/homebrew/opt/python@3.10/bin/python3.10: no such file or directory


In [5]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [7]:
n = int(0.9*len(data)) # first 90% of the data will be train , rest validation
train_data, val_data = data[:n], data[n:]

In [8]:
block_size= 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [12]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} then target is {target}")

when input is tensor([18]) then target is 47
when input is tensor([18, 47]) then target is 56
when input is tensor([18, 47, 56]) then target is 57
when input is tensor([18, 47, 56, 57]) then target is 58
when input is tensor([18, 47, 56, 57, 58]) then target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) then target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) then target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) then target is 58


In [14]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will be processed in parallel
block_size = 8 # what is the maximum context length of the predicitions ?

def get_batch(split):
    # generate a small batch of data of inputs x and target y 
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) 
    x= torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb, yb = get_batch('train')
print('inputs: ')
print(xb.shape)
print(xb)
print('targets: ')
print(yb.shape)
print(yb)

print('<--------------->')

for b in range(batch_size): # batch dimesion
    for t in range(block_size): # time dimesion
        context = xb[b,:t+1]
        target = yb[b,t]
        print(f"when input is {context} then target is {target}")

inputs: 
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets: 
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
<--------------->
when input is tensor([24]) then target is 43
when input is tensor([24, 43]) then target is 58
when input is tensor([24, 43, 58]) then target is 5
when input is tensor([24, 43, 58,  5]) then target is 57
when input is tensor([24, 43, 58,  5, 57]) then target is 1
when input is tensor([24, 43, 58,  5, 57,  1]) then target is 46
when input is tensor([24, 43, 58,  5, 57,  1, 46]) then target is 43
when input is tensor([24, 43, 58,  5, 57,  1, 46, 43]) then target is 39
when input is tensor([44]) then target is 53
when input is tensor([44, 53]) then target is 56
when input is tensor([4

In [15]:
print(xb) # our input to transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx is inputs aka xb
        # targets is outputs expected for training aka yb
        #  B is batch and T is chunk Size
        # idx and targets are both (B,T) tesnor of integers
        # B = Batch
        # T = Time aka Chunk Size
        # C= Channel(Vocab size) aka Embeeding Dimensions
        logits = self.token_embedding_table(idx)  # (B,T,C)
        #  Logits are teh score we determine for next charcters in the sequence

        # corss entropy accpets dimensions in different format
        # So get B, T, C from logits and do teh transformation
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
        #  logitcs are predictaions
        # This calculates the loss
        loss = F.cross_entropy(logits, targets)

        return logits, loss


model = BigramLanguageModel(vocab_size)
output, loss = model(xb, yb)
print(output.shape)
print(loss)

Embedding(65, 65)
torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tesnor of integers
        # B = Batch T = Time C= Channel(Vocab size)
        logits = self.token_embedding_table(idx) # (B,T,C)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            # print("B ", B, "T ", T, "C ", C)
            logits = logits.view(B*T, C)
            # print("logits shape ", logits.shape)
            # print("logits ", logits)
            targets = targets.view(B*T)
            # print("targets shape ", targets.shape)
            # print("targets ", targets)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step 
            # We are getting last one because that sthe predeication of what comes next 
            #  thats the reason we only consider last element
            # print('Generate logits.shape',logits.shape)
            logits = logits[:, -1, :] # becomes ( B, C)
            # print('Generate logits.shape After picking last ', logits.shape)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # becomes (B, C)
            # print(probs)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)# becomes (B,1)
            #append sampled index to the running sequence
            idx = torch.cat([idx, idx_next], dim=1) # becomes (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size) # vocab_size is length of all unique characters in input text data
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
idx = torch.zeros((batch_size, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)
tensor([[0.0091, 0.0071, 0.0053, 0.0030, 0.0141, 0.0078, 0.0197, 0.0081, 0.0109,
         0.0243, 0.0020, 0.0045, 0.0096, 0.0060, 0.0030, 0.0354, 0.0292, 0.0066,
         0.0101, 0.0199, 0.0010, 0.0124, 0.0335, 0.0137, 0.0086, 0.0016, 0.0024,
         0.0054, 0.0118, 0.0034, 0.0347, 0.0930, 0.0039, 0.0059, 0.0208, 0.0085,
         0.0089, 0.0235, 0.0024, 0.0056, 0.0046, 0.0030, 0.0131, 0.0017, 0.0023,
         0.0134, 0.0042, 0.0038, 0.0392, 0.0034, 0.0292, 0.0057, 0.0017, 0.0621,
         0.1199, 0.0013, 0.0323, 0.0017, 0.0172, 0.0061, 0.0165, 0.0351, 0.0379,
         0.0051, 0.0033],
        [0.0091, 0.0071, 0.0053, 0.0030, 0.0141, 0.0078, 0.0197, 0.0081, 0.0109,
         0.0243, 0.0020, 0.0045, 0.0096, 0.0060, 0.0030, 0.0354, 0.0292, 0.0066,
         0.0101, 0.0199, 0.0010, 0.0124, 0.0335, 0.0137, 0.0086, 0.0016, 0.0024,
         0.0054, 0.0118, 0.0034, 0.0347, 0.0930, 0.0039, 0.0059, 0.0208, 0.0085,
         0.0089, 0.

In [34]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [51]:
batch_size = 32
for steps in range(10000):
    # Sample a batch of data
    xb, yb = get_batch('train')
    # evaluate the loss
    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.4898500442504883


In [54]:
idx = torch.zeros((batch_size, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=500)[0].tolist()))


Y:
Asisl
Them. en anit:


Por try whals ty, ske ar.
OLIONUCI st; ivenses tcor ncham t puerd demy t, thatof he; tthiertusthisous, chokne, owhifo, w, ther t wes, he t hel fornd's pay;
AMI't spld; s a
NRY wantis, ime ou hag her,-t mak mor ovinthald, ntsedy bistre's ll m! mieathendim whay, s, why l,
Bulisis, anasioor whel a y d chin'llaithanoul lellartake:

JUCourit veeesprthotheng, tha atw; ouat ma d by INTORinibon re tulld cand s cher wnt peen s hellitrre, songharthofauser be, ctr hendaprag,
ARonc


**The mathematical trick in self-attention

In [55]:
# Conside the follwing toy example:

torch.manual_seed(1337)
B, T, C = 4,8,2 # batch , time, channels
x = torch.randn(B,T,C)
x. shape

torch.Size([4, 8, 2])

In [ ]:
# Version 1
# We want x[b, t] = mean_{i<=t} x[b,i]
# bow == bag of words
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        print('xprev', xprev)
        xbow[b,t] = torch.mean(xprev, 0)
        print('xbow', xbow)
print('xbow shape', xbow.shape)

xprev tensor([[ 0.1808, -0.0700]])
xbow tensor([[[ 0.1808, -0.0700],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        [[ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        [[ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]],

        [[ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.0000]]])
xprev tensor([[ 0.1808, 

In [79]:
# wei = weights
# Version 2
wei = torch.tril(torch.ones(T,T))
wei = wei /wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) -> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [81]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [88]:
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [80]:
# version 3 : Use Softmax

tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

In [61]:
torch.tril(torch.ones(3,3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [ ]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, dim=1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=', a)
print('b=', b) 
print('c=', c)

tensor([1., 2., 3.])
tensor([[1.],
        [2.],
        [3.]])
a= tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b= tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c= tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [98]:
# version 4: Self Attention!

torch.manual_seed(1337)
B, T, C = 4,8,32 # batch , time, channels
x = torch.randn(B,T,C)

# lets see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, head_size)
q = query(x) # (B, T, head_size)
wei = q @ k.transpose(-2,-1) # ( B, T , head_size) @ (B, head_size, T) -> (B, T, T)


tril = torch.tril(torch.ones(T,T))
# wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=1)
v = value(x) # (B, T, head_size)
out = wei @ v # (B, T, T) @ (B, T, head_size) -> (B, T, head_size)
# out = wei @ x
wei


tensor([[[0.0248, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0052, 0.0091, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0521, 0.0135, 0.2482, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3171, 0.0214, 0.1642, 0.1188, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0412, 0.0487, 0.1046, 0.0742, 0.2000, 0.0000, 0.0000, 0.0000],
         [0.1060, 0.5347, 0.2059, 0.1030, 0.7402, 0.0192, 0.0000, 0.0000],
         [0.4298, 0.3409, 0.1769, 0.2027, 0.0480, 0.8472, 0.2329, 0.0000],
         [0.0238, 0.0316, 0.1002, 0.5013, 0.0117, 0.1336, 0.7671, 1.0000]],

        [[0.0443, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0042, 0.0375, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0560, 0.0210, 0.2496, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3679, 0.1441, 0.4929, 0.0438, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0088, 0.1052, 0.0604, 0.5847, 0.2046, 0.0000, 0.0000, 0.0000],
         [0.0367, 0.089

In [96]:
wei[0]

tensor([[0.0248, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0052, 0.0091, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0521, 0.0135, 0.2482, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3171, 0.0214, 0.1642, 0.1188, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0412, 0.0487, 0.1046, 0.0742, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1060, 0.5347, 0.2059, 0.1030, 0.7402, 0.0192, 0.0000, 0.0000],
        [0.4298, 0.3409, 0.1769, 0.2027, 0.0480, 0.8472, 0.2329, 0.0000],
        [0.0238, 0.0316, 0.1002, 0.5013, 0.0117, 0.1336, 0.7671, 1.0000]],
       grad_fn=<SelectBackward0>)